# References

https://www.kaggle.com/code/mtszkw/technical-indicators-for-trading-stocks
https://www.kaggle.com/code/thebrownviking20/everything-you-can-do-with-a-time-series
https://www.diva-portal.org/smash/get/diva2:1775077/FULLTEXT01.pdf
https://pub.towardsai.net/predicting-stock-prices-using-arima-fourier-transforms-and-technical-indicators-with-deep-43a164859683


## Import dependencies

In [14]:
! pip install -r requirements.txt


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
import pandas as pd

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly as py
import plotly.io as pio
import plotly.graph_objs as go
from plotly.subplots import make_subplots

import numpy as np                                    # For matrices!
import matplotlib.pyplot as plt                       # To visualize
from sklearn.linear_model import LinearRegression     # For the regression itself
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn import svm
from sklearn.metrics import mean_squared_error, f1_score
from sklearn.preprocessing import OneHotEncoder       # To convert discrete strings to vectors!
from sklearn.preprocessing import normalize           # For normalizing
import seaborn as sns                                 # For plots

In [3]:
# Show charts when running kernel
init_notebook_mode(connected=True)

# Change default background color for all visualizations
layout=go.Layout(paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(250,250,250,0.8)')
fig = go.Figure(layout=layout)
templated_fig = pio.to_templated(fig)
pio.templates['my_template'] = templated_fig.layout.template
pio.templates.default = 'my_template'

In [4]:
# SnP500
! gdown --id 1KveL-W2L6YxO-1NKn0n6FeGTKLWgvAL4
! gdown --id 1zpiNfA7BXiHjqj1tOpNnC3lRdDfZ8kC3
! gdown --id 1zdvABHAhe3pmWE2p4UWUpKJN-1q6fVG7
! gdown --id 1rtaZZ9w4kZwuXqcU2QDZq4rD6CHm7fMW

# Nasdaq
! gdown --id 1NyHRoU2YiaCKIhy7afUTuAmC0haWK1Ny
! gdown --id 1vBAdm8bRRD6jZvRbbQ3q4t5hs19fadke
! gdown --id 1pX1bRRoQyxNty5CF6f3BDDeBVFRAd2iQ
! gdown --id 1ngj9kV06RK2ztU5trUtLKy_q2U9ZpYvZ

C:\Users\rishi\anaconda3\lib\site-packages\gdown\cli.py:126: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (uriginal): https://drive.google.com/uc?id=1KveL-W2L6YxO-1NKn0n6FeGTKLWgvAL4
From (redirected): https://drive.google.com/uc?id=1KveL-W2L6YxO-1NKn0n6FeGTKLWgvAL4&confirm=t&uuid=00241788-0750-4dd7-9c18-0d006377d850
To: C:\Users\rishi\AppData\Local\Programs\Microsoft VS Code\ES_continuous_adjusted_1min_2011_2018.txt

  0%|          | 0.00/158M [00:00<?, ?B/s]
  0%|          | 524k/158M [00:00<01:37, 1.61MB/s]
  1%|          | 1.05M/158M [00:00<01:01, 2.55MB/s]
  1%|          | 1.57M/158M [00:00<00:51, 3.05MB/s]
  1%|▏         | 2.10M/158M [00:00<00:44, 3.53MB/s]
  2%|▏         | 2.62M/158M [00:00<00:40, 3.83MB/s]
  2%|▏         | 3.15M/158M [00:00<00:37, 4.12MB/s]
  2%|▏         | 3.67M/158M [00:01<00:35, 4.30MB/s]
  3%|▎         | 4.19M/158M [00:01<00:3

In [7]:
pd.read_csv('ES_continuous_adjusted_1min_2011_2018.txt', encoding='latin-1')
snp_raw_data = pd.read_csv('ES_continuous_adjusted_5min_2011_2018.txt', encoding='latin-1')
pd.read_csv('ES_continuous_adjusted_30min_2011_2018.txt', encoding='latin-1')
pd.read_csv('ES_continuous_adjusted_1hour_2011_2018.txt', encoding='latin-1')

pd.read_csv('NQ_continuous_adjusted_1min_2011_2018.txt', encoding='latin-1')
ndq_raw_data= pd.read_csv('NQ_continuous_adjusted_5min_2011_2018.txt', encoding='latin-1')
pd.read_csv('NQ_continuous_adjusted_30min_2011_2018.txt', encoding='latin-1')
pd.read_csv('NQ_continuous_adjusted_1hour_2011_2018.txt', encoding='latin-1')

,DateTime,Open,High,Low,Close,Volume
0,2011-01-02 18:00:00,2361.25,2365.00,2360.50,2362.75,1127
1,2011-01-02 19:00:00,2363.00,2364.25,2361.00,2363.50,191
2,2011-01-02 20:00:00,2363.50,2364.00,2362.75,2362.75,207
3,2011-01-02 21:00:00,2362.75,2364.50,2362.50,2364.50,255
4,2011-01-02 22:00:00,2364.50,2369.50,2364.25,2368.25,667
...,...,...,...,...,...,...
48176,2018-12-31 12:00:00,6464.50,6487.50,6451.25,6479.75,33178
48177,2018-12-31 13:00:00,6479.75,6501.50,6461.75,6465.00,22229
48178,2018-12-31 14:00:00,6464.75,6477.25,6436.50,6458.75,33198
48179,2018-12-31 15:00:00,6458.50,6485.25,6433.75,6484.75,55508


In [10]:
snp_raw_data.head()

,DateTime,Open,High,Low,Close,Volume
0,2011-01-02 18:00:00,1056.75,1058.50,1056.00,1058.00,3522
1,2011-01-02 18:05:00,1058.00,1058.00,1057.00,1057.25,777
2,2011-01-02 18:10:00,1057.25,1057.25,1056.75,1057.00,511
3,2011-01-02 18:15:00,1057.25,1057.50,1057.00,1057.50,300
4,2011-01-02 18:20:00,1057.50,1057.50,1056.75,1057.00,169


In [9]:
ndq_raw_data.head()

,DateTime,Open,High,Low,Close,Volume
0,2011-01-02 18:00:00,2361.25,2365.00,2361.25,2364.25,350
1,2011-01-02 18:05:00,2364.25,2364.50,2361.75,2362.25,188
2,2011-01-02 18:10:00,2362.25,2362.25,2361.50,2362.00,70
3,2011-01-02 18:15:00,2362.00,2362.00,2361.50,2362.00,147
4,2011-01-02 18:20:00,2362.00,2362.00,2360.75,2361.00,111
